In [7]:
import pandas as pds

data_uri='./data/ZINC_310k.csv'

kk = pds.read_csv(data_uri).as_matrix()

/home/gcolmenarejo/anaconda3/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [16]:
kk[1:5,]

array([['COc1ccc(N2CC(C(=O)Oc3cc(C)ccc3C)CC2=O)cc1', 339.1470582,
        3.27054, 0.633604683],
       ['CC1CCC(C(N)=O)CN1C(=O)Nc1cnn(C(C)(C)C)c1', 307.200825, 1.7558,
        0.871419657],
       ['CN(Cc1nc2ccccc2c(=O)[nH]1)C(=O)c1cn2ccccc2n1', 333.1225747,
        1.8430000000000002, 0.620005175],
       ['CC(C)c1noc(C(C)N2CCN(CC(=O)N(C)C)CC2)n1', 309.2164751, 0.9598,
        0.807001593]], dtype=object)

In [36]:
import random
#random.sample(range(1,10),10
random.seed(123)
id = random.sample(range(0, 100), 100)

In [41]:
id[1:10]
kk[0,]

array(['COc1cc(Cl)cc2c1OC(C)C(=O)N(c1c(F)cccc1F)C2', 353.06302739999995,
       3.9409, 0.8204996929999999], dtype=object)

In [44]:
import time

# Start time count
start_time = time.time()


# End time count
elapsed_time = time.time() - start_time
print(elapsed_time)
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

4.1484832763671875e-05
00:00:00


::: data preparation
::: model training


/home/gcolmenarejo/anaconda3/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py:34: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/gcolmenarejo/anaconda3/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py:35: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


ValueError: Variable rnnpredictor/bidirectional_rnn/fw/multi_rnn_cell/cell_0/gru_cell/gates/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/gcolmenarejo/cmd/SSVAE.py", line 304, in _rnnpredictor
    _, final_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, x, initial_state_fw=init_state_fw, initial_state_bw=init_state_bw)
  File "/home/gcolmenarejo/cmd/SSVAE.py", line 26, in __init__
    self.classifier_L_out = self._rnnpredictor(self.x_L, self.dim_h, 2*self.dim_y, reuse = False)
  File "<ipython-input-1-45a7d1ee8926>", line 88, in <module>
    n_hidden = n_hidden, batch_size = batch_size, beta = float(beta), char_set = char_set)


In [57]:
id1 = 50000
id2 = 300000
for i in range(id1,id2+1,id1):
    print(i)


50000
100000
150000
200000
250000
300000


In [7]:
smis = []

In [4]:
import csv

In [2]:
smi

'CC1ccccccccc(CC))CCCCCCCCCCCCCCCCCCCCCCCCCCCC1'

In [5]:
for t in range(10):
        smis[t] = model.sampling_unconditional()

RuntimeError: Attempted to use a closed Session.

In [8]:
smis.append("c1ccccc1")
smis.append("n1ccccc1")

In [9]:
smis

['c1ccccc1', 'n1ccccc1']

In [10]:
csvfile = "kk.csv"

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in smis:
        writer.writerow([val]) 

In [18]:
smi

'CC1ccccccccc(CC))CCCCCCCCCCCCCCCCCCCCCCCCCCCC1'

In [19]:
smis

['c1ccccc1', 'n1ccccc1']

In [20]:
smis.append(smi)

In [21]:
smis

['c1ccccc1', 'n1ccccc1', 'CC1ccccccccc(CC))CCCCCCCCCCCCCCCCCCCCCCCCCCCC1']

In [25]:
"hola"+str(10000)+".csv"

'hola10000.csv'

In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pds
from preprocessing import smiles_to_seq, vectorize
import SSVAE
import time
import csv

from preprocessing import get_property, canonocalize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error


# Start time count
start_time = time.time()


# pre-defined parameters
frac=0.5
beta=10000.
char_set=[' ','1','2','3','4','5','6','7','8','9','-','#','(',')','[',']','+','=','B','Br','c','C','Cl','F','H','I','N','n','O','o','P','p','S','s','Si','Sn']
data_uri='./data/zinc1100.csv'
save_uri='./zinc_model1100.ckpt'

ntrn=1000
frac_val=0.05
ntst=100


# data preparation
print('::: data preparation')

smiles = pds.read_csv(data_uri).as_matrix()[:ntrn+ntst,0] #0: SMILES
Y = np.asarray(pds.read_csv(data_uri).as_matrix()[:ntrn+ntst,1:], dtype=np.float32) # 1: MolWT, 2: LogP, 3: QED 

list_seq = smiles_to_seq(smiles, char_set)
Xs, X=vectorize(list_seq, char_set)

tstX=X[-ntst:]
tstXs=Xs[-ntst:]
tstY=Y[-ntst:]


for n in range(500,1001,500):
    ntrn = n

    X=X[:ntrn]
    Xs=Xs[:ntrn]
    Y=Y[:ntrn]

    nL=int(len(Y)*frac)
    nU=len(Y)-nL
    nL_trn=int(nL*(1-frac_val))
    nL_val=nL-nL_trn
    nU_trn=int(nU*(1-frac_val))
    nU_val=nU-nU_trn
    perm_id=np.random.permutation(len(Y))

    trnX_L=X[perm_id[:nL_trn]]
    trnXs_L=Xs[perm_id[:nL_trn]]
    trnY_L=Y[perm_id[:nL_trn]]

    valX_L=X[perm_id[nL_trn:nL_trn+nL_val]]
    valXs_L=Xs[perm_id[nL_trn:nL_trn+nL_val]]
    valY_L=Y[perm_id[nL_trn:nL_trn+nL_val]]

    trnX_U=X[perm_id[nL_trn+nL_val:nL_trn+nL_val+nU_trn]]
    trnXs_U=Xs[perm_id[nL_trn+nL_val:nL_trn+nL_val+nU_trn]]

    valX_U=X[perm_id[nL_trn+nL_val+nU_trn:]]
    valXs_U=Xs[perm_id[nL_trn+nL_val+nU_trn:]]

    scaler_Y = StandardScaler()
    scaler_Y.fit(Y)
    trnY_L=scaler_Y.transform(trnY_L)
    valY_L=scaler_Y.transform(valY_L)


    ## model training
    print('::: model training, n=', str(n))

    seqlen_x = X.shape[1]
    dim_x = X.shape[2]
    dim_y = Y.shape[1]
    dim_z = 100
    dim_h = 250

    n_hidden = 3
    batch_size = 200

    model = SSVAE.Model(seqlen_x = seqlen_x, dim_x = dim_x, dim_y = dim_y, dim_z = dim_z, dim_h = dim_h,
                        n_hidden = n_hidden, batch_size = batch_size, beta = float(beta), char_set = char_set)

    with model.session:
        model.train(trnX_L=trnX_L, trnXs_L=trnXs_L, trnY_L=trnY_L, trnX_U=trnX_U, trnXs_U=trnXs_U,
                    valX_L=valX_L, valXs_L=valXs_L, valY_L=valY_L, valX_U=valX_U, valXs_U=valXs_U)
        model.saver.save(model.session, save_uri)

        ## property prediction performance
        tstY_hat=scaler_Y.inverse_transform(model.predict(tstX))

        for j in range(dim_y):
            print([j, mean_absolute_error(tstY[:,j], tstY_hat[:,j])])
        
        
        ## unconditional generation
        smis_u = []
        for t in range(10):
            smi = model.sampling_unconditional()
            smis_u.append(smi)
          
        csvfile = "exp1/unc"+str(n)+".csv"
        with open(csvfile, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            for val in smis_u:
                writer.writerow([val])
        writer.close()
     
        ## conditional generation (e.g. MolWt=250)
        smis_c = []
        yid = 0
        ytarget = 250.
        ytarget_transform = (ytarget-scaler_Y.mean_[yid])/np.sqrt(scaler_Y.var_[yid])
    
        for t in range(10):
            smi = model.sampling_conditional(yid, ytarget_transform)
            smis_c.append(smi)
    
        csvfile = "exp1/con"+str(n)+".csv"
        with open(csvfile, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            for val in smis_c:
                writer.writerow([val])
        writer.close()
        
# End time count
elapsed_time = time.time() - start_time
print(elapsed_time)
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

In [26]:
smis_u = ["cc","acavb","asdf"]
smis_c = ["aas","albnd","lanb"]

csvfile = "shit1.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in smis_u:
        writer.writerow([val]) 

In [30]:
csvfile = "exp1/shit2.csv"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in smis_c:
        writer.writerow([val]) 
        

In [29]:
help(csv.close)

AttributeError: module 'csv' has no attribute 'close'

In [31]:
85200/31

2748.3870967741937

In [32]:
85200-2748

82452

In [36]:
for i in [50000,82452]:
    print(i)

50000
82452


In [37]:
2748+82452

85200